#Лабораторна робота 7

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 488.2 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357297 sha256=ad835bc020900b6993379955bcc27ec95ceb2cd908c14eabaccc2637c061f11c
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету:")
for row in data.raw_ratings[:5]:
    print(row)

Перші 5 рядків датасету:
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [ ]:
data = Dataset.load_builtin('ml-100k')

print("Перші 5 рядків датасету з поясненням:")
for row in data.raw_ratings[:5]:
    user_id, movie_id, rating, timestamp = row
    print(f"Користувач {user_id} оцінив фільм {movie_id} на {rating} балів (часова позначка: {timestamp})")

Перші 5 рядків датасету з поясненням:
Користувач 196 оцінив фільм 242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм 302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм 377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм 51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм 346 на 1.0 балів (часова позначка: 886397596)


In [ ]:
trainset, testset = train_test_split(data, test_size=0.2)

# Параметри для підбору через GridSearchCV для SVD
param_grid_svd = {
    'n_factors': [50, 100],
    'lr_all': [0.005, 0.010],
    'reg_all': [0.02, 0.05]
}

# Параметри для підбору через GridSearchCV для KNNBasic
param_grid_knn = {
    'k': [20, 40],
    'min_k': [1, 5],
    'sim_options': {
        'name': ['cosine', 'pearson'],
        'user_based': [False, True]
    }
}

grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [ ]:
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

# Вибір моделі з найменшим MAE
if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 0.7246773447231387
Найкращий MAE для KNNBasic: 0.8030069352347221
Вибраний алгоритм: SVD


In [ ]:
best_model.fit(trainset)

# Виведення рекомендацій для конкретного користувача
user_id = str(22)  # ID користувача для якого ми хочемо отримати рекомендації
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

# Отримання рекомендацій
all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 22: 102
Топ-10 фільмів, рекомендованих для користувача:
Фільм 100 з прогнозованим рейтингом 5.00
Фільм 183 з прогнозованим рейтингом 4.86
Фільм 12 з прогнозованим рейтингом 4.80
Фільм 302 з прогнозованим рейтингом 4.75
Фільм 50 з прогнозованим рейтингом 4.73
Фільм 64 з прогнозованим рейтингом 4.69
Фільм 357 з прогнозованим рейтингом 4.68
Фільм 272 з прогнозованим рейтингом 4.65
Фільм 1449 з прогнозованим рейтингом 4.62
Фільм 98 з прогнозованим рейтингом 4.60


Висновок - У даному завданні було створено рекомендаційну систему на основі датасету ml-100k, використовуючи бібліотеку Surprise. Завантажила дані,що дозволили ознайомитися зі структурою. Реалізувала два алгоритми для рекомендаційної системи: SVD та KNNBasic.  Результати показали, що найкращий MAE для SVD становив 0.7247, тоді як для KNNBasic — 0.8030. На основі цих показників було обрано SVD як найкращий алгоритм для даної задачі. Також система надала 10 найкращих фільмів, рекомендованих для перегляду, з урахуванням уподобань користувача.

Завдання 2

In [ ]:
# Встановлення бібліотеки Surprise
!pip install scikit-surprise

In [ ]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy

In [ ]:
# Завантаження датасету ml-100k
data = Dataset.load_builtin('ml-100k')

# Розподіл даних на тренувальний та тестовий набори
trainset, testset = train_test_split(data, test_size=0.2)

# Створення моделі SVD
model = SVD()

# Тренування моделі
model.fit(trainset)

# Оцінка моделі
predictions = model.test(testset)

# Оцінювання моделі з використанням RMSE та MAE
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.9281
MAE:  0.7326


In [ ]:
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

RMSE: 0.93
MAE: 0.73


In [ ]:
# Отримання рекомендацій для конкретного користувача
user_id = str(22)  # ID користувача для якого ми хочемо отримати рекомендації

# Отримання всіх фільмів
all_items = set(trainset.all_items())

# Отримання рецензій користувача
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

# Прогнозування рейтингів для нерецензованих фільмів
predictions = [(item, model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

In [ ]:
# Виведення 10 найкращих фільмів
print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Топ-10 фільмів, рекомендованих для користувача:
Фільм 318 з прогнозованим рейтингом 4.77
Фільм 64 з прогнозованим рейтингом 4.65
Фільм 98 з прогнозованим рейтингом 4.59
Фільм 408 з прогнозованим рейтингом 4.56
Фільм 170 з прогнозованим рейтингом 4.56
Фільм 479 з прогнозованим рейтингом 4.53
Фільм 525 з прогнозованим рейтингом 4.52
Фільм 22 з прогнозованим рейтингом 4.50
Фільм 427 з прогнозованим рейтингом 4.49
Фільм 178 з прогнозованим рейтингом 4.49


Висновок - У даному завданні була побудована рекомендаційна система, що використовує бібліотеку Surprise, на основі датасету ml-100k.Використання RMSE та MAE дозволило оцінити точність моделі. Отримані значення свідчать про адекватність моделі в прогнозуванні рейтингів. Чим нижчі значення, тим вища точність прогнозів. Система згенерувала список з 10 рекомендованих фільмів для конкретного користувача. Ці рекомендації формуються на основі вже виставлених рейтингів користувача